In [3]:
import pandas as pd
import numpy as np
import os
import random
import collections

In [27]:
parquet_path = './refinement_ETD_dont_improve_out_2/data/dlomix/refinement_dataset/processed_dataset.parquet'

In [28]:
ds = pd.read_parquet(parquet_path)

In [4]:
len(ds)

61493

In [6]:
ds.columns

Index(['modified_sequence', 'precursor_charge_onehot',
       'collision_energy_aligned_normed', 'method_nbr', 'intensities_raw',
       'sequence', 'raw_file', 'scan_number'],
      dtype='object')

In [7]:
ds.head()

,modified_sequence,precursor_charge_onehot,collision_energy_aligned_normed,method_nbr,intensities_raw,sequence,raw_file,scan_number
0,AAVLERLVDI,"[0, 1, 0, 0, 0, 0]",35,3,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.0, 0.0, -1....",AAVLERLVDI,02445b_BA8-TUM_HLA_8_01_01-ETD-1h-R1,12242
1,AAVLERLVDI,"[0, 1, 0, 0, 0, 0]",35,3,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.0, 0.0, -1....",AAVLERLVDI,02445b_BA8-TUM_HLA_8_01_01-ETD-1h-R1,12252
2,AAVLERLVDI,"[0, 1, 0, 0, 0, 0]",35,3,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.0, 0.0, -1....",AAVLERLVDI,02445b_BA8-TUM_HLA_8_01_01-ETD-1h-R1,12265
3,AAVLERLVDI,"[0, 1, 0, 0, 0, 0]",35,3,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.0, 0.0, -1....",AAVLERLVDI,02445b_BA8-TUM_HLA_8_01_01-ETD-1h-R1,12278
4,AAVLERLVDI,"[0, 1, 0, 0, 0, 0]",35,3,"[0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.0, 0.0, -1....",AAVLERLVDI,02445b_BA8-TUM_HLA_8_01_01-ETD-1h-R1,12294


In [9]:
def argshuffle_splits(idx, traininsplit):
    train_idx = list(np.random.permutation(idx[:traininsplit]))
    test_idx = list(np.random.permutation(idx[traininsplit:]))
    return train_idx, test_idx

def peptide_argsort(sequence_integer, seed=42):
    random.seed(seed)
    peptide_groups = collections.defaultdict(list)
    for index, row in enumerate(sequence_integer):
        peptide_groups[tuple(row)].append(index)

    # shuffle within peptides
    for indeces in peptide_groups.values():
        random.shuffle(indeces)

    # shuffle peptides
    peptides = list(peptide_groups.keys())
    random.shuffle(peptides)

    # join indeces
    indeces = []
    for peptide in peptides:
        indeces.extend(peptide_groups[peptide])
    return indeces

In [15]:
sequence = ds['modified_sequence']
n = sequence.shape[0]

In [17]:
n_split = int(n * float(0.7))
print("training split at: ", n_split, "/", n)
print("shuffling peptides")
idx = peptide_argsort(sequence, seed=42)

training split at:  43045 / 61493
shuffling peptides


In [19]:
print("shuffling train and test")
train_idx, test_idx = argshuffle_splits(idx, n_split)

shuffling train and test


In [20]:
len(train_idx), len(test_idx)

(43045, 18448)

In [22]:
ds_train = ds.iloc[train_idx, :]
ds_val = ds.iloc[test_idx, :]

In [24]:
train_seqs = set(ds_train['modified_sequence'])
val_seqs = set(ds_val['modified_sequence'])

In [1]:
from dlomix.interface import process_dataset, load_keras_model
import os

2024-08-20 15:59:14.691458: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 15:59:14.691501: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 15:59:14.693221: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-20 15:59:14.701680: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-20 15:59:17.235025: W tensorflow/compiler/tf2

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [3]:
model = load_keras_model('baseline')

2024-08-20 15:59:22.953719: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7297 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:82:00.0, compute capability: 6.1


In [4]:
proc_ds = process_dataset(
    parquet_file_path='/cmnfs/data/proteomics/Prosit_PTMs/TUM_mod_monomethyl.parquet',
    model=model,
    modifications=['K[UNIMOD:34]', 'R[UNIMOD:34]']
)


            There are new tokens in the dataset, which are not supported by the loaded model.
            Either load a different model or transfer learning needs to be done.
            
Casting the dataset: 100%|██████████| 67096/67096 [00:12<00:00, 5492.52 examples/s]


In [11]:
train_seqs = [''.join(seq) for seq in proc_ds['train']['_parsed_sequence']]

In [12]:
val_seqs = [''.join(seq) for seq in proc_ds['val']['_parsed_sequence']]

In [14]:
len(train_seqs), len(val_seqs)

(268383, 67096)

In [15]:
len(set(train_seqs) & set(val_seqs))

1

### -> works 